In [2]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

from gurobi_ml import add_predictor_constr
import gurobipy_pandas as gppd

In [6]:
# Get the data

data_url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/price_optimization/"
avocado = pd.read_csv(
    data_url + "HABdata_2019_2022.csv"
)  # dataset downloaded directly from HAB
avocado_old = pd.read_csv(
    data_url + "kaggledata_till2018.csv"
)  # dataset downloaded from Kaggle
avocado = pd.concat([avocado, avocado_old])

# Add the index for each year from 2015 through 2022
avocado["date"] = pd.to_datetime(avocado["date"])
avocado["year"] = pd.DatetimeIndex(avocado["date"]).year
avocado["year_index"] = avocado["year"] - 2015
avocado = avocado.sort_values(by="date")

# Define the peak season
avocado["month"] = pd.DatetimeIndex(avocado["date"]).month
peak_months = range(2, 8)  # <--------- Set the months for the "peak season"


def peak_season(row):
    return 1 if int(row["month"]) in peak_months else 0


avocado["peak"] = avocado.apply(lambda row: peak_season(row), axis=1)

# Scale the number of avocados to millions
avocado["units_sold"] = avocado["units_sold"] / 1000000

# Select only conventional avocados
avocado = avocado[avocado["type"] == "Conventional"]

avocado = avocado[
    ["date", "units_sold", "price", "region", "year", "month", "year_index", "peak"]
].reset_index(drop=True)

avocado

C:\Users\RC\AppData\Local\Temp\ipykernel_25880\3464000544.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  avocado["date"] = pd.to_datetime(avocado["date"])


,date,units_sold,price,region,year,month,year_index,peak
0,2015-01-04,3.382800,1.020000,Great_Lakes,2015,1,0,0
1,2015-01-04,2.578275,1.100000,Midsouth,2015,1,0,0
2,2015-01-04,5.794411,0.890000,West,2015,1,0,0
3,2015-01-04,3.204112,0.980000,Southeast,2015,1,0,0
4,2015-01-04,0.321824,1.050000,Northern_New_England,2015,1,0,0
...,...,...,...,...,...,...,...,...
3397,2022-05-15,4.150433,1.269883,SouthCentral,2022,5,7,1
3398,2022-05-15,4.668815,1.644873,Northeast,2022,5,7,1
3399,2022-05-15,32.745321,1.527357,Total_US,2022,5,7,1
3400,2022-05-15,3.542902,1.514583,Midsouth,2022,5,7,1


In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

feat_transform = make_column_transformer(
    (OneHotEncoder(drop="first"), ["region"]),
    (StandardScaler(), ["price"]),
    ("passthrough", ["peak"]),
    verbose_feature_names_out=False,
    remainder='drop'
)


regions = [
    "Great_Lakes",
    "Midsouth",
    "Northeast",
    "Northern_New_England",
    "SouthCentral",
    "Southeast",
    "West",
    "Plains",
]
df = avocado[avocado.region.isin(regions)]

X = df[["region", "price", "peak"]]
y = df["units_sold"]

In [10]:
X

,region,price,peak
0,Great_Lakes,1.020000,0
1,Midsouth,1.100000,0
2,West,0.890000,0
3,Southeast,0.980000,0
4,Northern_New_England,1.050000,0
...,...,...,...
3396,Northern_New_England,1.513707,1
3397,SouthCentral,1.269883,1
3398,Northeast,1.644873,1
3400,Midsouth,1.514583,1


In [12]:
y

0       3.382800
1       2.578275
2       5.794411
3       3.204112
4       0.321824
          ...   
3396    0.445830
3397    4.150433
3398    4.668815
3400    3.542902
3401    1.560202
Name: units_sold, Length: 3024, dtype: float64

In [14]:
from sklearn.model_selection import train_test_split

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

In [16]:

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import clone
from xgboost import XGBRegressor
from time import time
args={"random_state":1}
regressions = {"Linear Regression": {"regressor":LinearRegression() },
               "MLP Regression": {"regressor": MLPRegressor([8]*2, max_iter=1000, **args)},
               "Decision Tree": {"regressor": DecisionTreeRegressor(max_leaf_nodes=50, **args)},
               "Random Forest": {"regressor": RandomForestRegressor(n_estimators=10, max_leaf_nodes=100, **args)},
               "Gradient Boosting":
               {"regressor" : GradientBoostingRegressor(n_estimators=20, **args)},
              "XGB Regressor": {"regressor": XGBRegressor(n_estimators=20, **args)}}

# Add polynomial features for linear regression and MLP
regressions_poly = {}
for regression in ["Linear Regression", "MLP Regression"]:
    data = {"regressor": (PolynomialFeatures(),clone(regressions[regression]["regressor"]))}
    regressions_poly[f"{regression} polynomial feats"] = data
# Merge dictionary of polynomial features
regressions |= regressions_poly

In [18]:
for regression, data in regressions.items():
    regressor = data["regressor"]
    if isinstance(regressor, tuple):
        lin_reg = make_pipeline(feat_transform,
                                *regressor)
    else:
        lin_reg = make_pipeline(feat_transform,
                                regressor)
    train_start = time()
    lin_reg.fit(X_train, y_train)
    data[("Learning","time")] = time() - train_start
    data["pipeline"] = lin_reg

    # Get R^2 from test data
    y_pred = lin_reg.predict(X_test)
    r2_test = r2_score(y_test, y_pred)
    y_pred = lin_reg.predict(X_train)
    r2_train = r2_score(y_train, y_pred)
    data[("Learning", "R2 test")] = r2_test
    data[("Learning", "R2 train")] = r2_train
    print(f"{regression:<18} R^2 value in the test set is {r2_test:.3f} training {r2_train:.3f}")

Linear Regression  R^2 value in the test set is 0.868 training 0.882
MLP Regression     R^2 value in the test set is 0.883 training 0.894
Decision Tree      R^2 value in the test set is 0.882 training 0.906
Random Forest      R^2 value in the test set is 0.874 training 0.924
Gradient Boosting  R^2 value in the test set is 0.802 training 0.829
XGB Regressor      R^2 value in the test set is 0.887 training 0.921
Linear Regression polynomial feats R^2 value in the test set is 0.883 training 0.892
MLP Regression polynomial feats R^2 value in the test set is 0.887 training 0.897


In [22]:
# Sets and parameters
B = 30  # total amount ot avocado supply

peak_or_not = 1  # 1 if it is the peak season; 1 if isn't

c_waste = 0.1  # the cost ($) of wasting an avocado
# the cost of transporting an avocado
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4,
        "Northern_New_England": 0.5,
        "SouthCentral": 0.3,
        "Southeast": 0.2,
        "West": 0.2,
        "Plains": 0.2,
    }, name='transport_cost'
)

c_transport = c_transport.loc[regions]
# the cost of transporting an avocado

# Get the lower and upper bounds from the dataset for the price and the number of products to be stocked
a_min = 0  # minimum avocado price in each region
a_max = 2  # maximum avocado price in each region

data = pd.concat([c_transport,
                  df.groupby("region")["units_sold"].min().rename('min_delivery'),
                  df.groupby("region")["units_sold"].max().rename('max_delivery')], axis=1)


In [24]:
data

,transport_cost,min_delivery,max_delivery
Great_Lakes,0.3,2.063574,7.094765
Midsouth,0.1,1.845443,6.168572
Northeast,0.4,2.364424,8.836406
Northern_New_England,0.5,0.219690,0.917984
SouthCentral,0.3,3.687130,10.323175
Southeast,0.2,2.197764,7.810475
West,0.2,3.260102,11.274749
Plains,0.2,1.058938,3.575499


In [26]:
m = gp.Model("Avocado_Price_Allocation")

p = gppd.add_vars(m, data, name="price", lb=a_min, ub=a_max)
d = gppd.add_vars(m, data, name="demand") # Add variables for the regression
w = m.addVar(name="w") # excess wasteage
m.update()

m.setObjective((p * d).sum() - c_waste * w - (c_transport * d).sum())
m.ModelSense = GRB.MAXIMIZE

m.addConstr(d.sum() + w == B)
m.update()

Restricted license - for non-production use only - expires 2026-11-23


In [28]:
feats = pd.DataFrame(
    data={
        "peak": peak_or_not,
        "region": regions,
        "price": p
    },
    index=regions
)
feats = feats[["region", "price", "peak"]]

In [30]:
for regression, data in regressions.items():
    pred_constr = add_predictor_constr(m, data["pipeline"], feats, d, epsilon=1e-5)

    pred_constr.print_stats()

    data[("Optimization", "#constrs")] = m.NumConstrs + m.NumQConstrs + m.NumGenConstrs
    data[("Optimization", "#vars")] = m.NumVars
    m.Params.NonConvex = 2
    m.Params.OutputFlag = 0
    try:
        start = time()
        m.optimize()
        data[("Optimization", "time")] = time() - start
        data[("Optimization", "value")] = m.ObjVal
        data[("Optimization", "viol")] = m.MaxVio
        data[("Optimization", "error")] = pred_constr.get_error().max()
    except:
        data[("Optimization", "value")] = float('nan')
        data[("Optimization", "viol")] = float('nan')
        data[("Optimization", "error")] = float('nan')
        break
        pass
    pred_constr.remove()

Model for pipe:
72 variables
16 constraints
Input has shape (8, 3)
Output has shape (8, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
col_trans             (8, 9)            8            8            0            0

lin_reg               (8, 1)           64            8            0            0

--------------------------------------------------------------------------------
Set parameter NonConvex to value 2
Model for pipe0:
328 variables
144 constraints
128 general constraints
Input has shape (8, 3)
Output has shape (8, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    

In [34]:
res = pd.DataFrame.from_dict(regressions, orient='index').drop(["regressor", "pipeline"], axis=1)

In [36]:
res.columns = pd.MultiIndex.from_tuples(res.columns)

In [38]:
res.round(3)

Learning                  Optimization  \
                                       time R2 test R2 train     #constrs   
Linear Regression                     0.053   0.868    0.882         17.0   
MLP Regression                        1.667   0.883    0.894        273.0   
Decision Tree                         0.022   0.882    0.906          NaN   
Random Forest                         0.090   0.874    0.924          NaN   
Gradient Boosting                     0.056   0.802    0.829          NaN   
XGB Regressor                         0.071   0.887    0.921          NaN   
Linear Regression polynomial feats    0.039   0.883    0.892          NaN   
MLP Regression polynomial feats       1.807   0.887    0.897          NaN   

                                                                     
                                    #vars   time   value viol error  
Linear Regression                    89.0  0.124  36.715  0.0   0.0  
MLP Regression                      345.0    NaN     NaN  NaN   NaN  
Decision Tree                         NaN    NaN     NaN  NaN   NaN  
Random Forest                         NaN    NaN     NaN  NaN   NaN  
Gradient Boosting                     NaN    NaN     NaN  NaN   NaN  
XGB Regressor                         NaN    NaN     NaN  NaN   NaN  
Linear Regression polynomial feats    NaN    NaN     NaN  NaN   NaN  
MLP Regression polynomial feats       NaN    NaN     NaN  NaN   NaN